In [ ]:
'''
利用pypylon以及opencv共同实现的一个实时Demo，
会在棋盘格的对应位置上绘制出小立方体，
具体效果请见文件夹中的演示视频
'''

from pypylon import pylon
import cv2
import cv2
import numpy as np
import glob

def draw_lines(img, points):
    pairs = []
    pairs += [(i, i+4) for i in range(4)]
    pairs += [(i, (i+1)%4) for i in range(4)]
    pairs += [(i+4, (i+1)%4 + 4) for i in range(4)]
    for tup in pairs:
        pt1 = (int(points[tup[0], 0]), int(points[tup[0], 1]))
        pt2 = (int(points[tup[1], 0]), int(points[tup[1], 1]))
        cv2.line(img, pt1, pt2, (0, 0, 255), 4, 3)

    cv2.imwrite('1.png', img)

# 设置寻找亚像素角点的参数，采用的停止准则是最大循环次数30和最大误差容限0.001
criteria = (cv2.TERM_CRITERIA_MAX_ITER | cv2.TERM_CRITERIA_EPS, 30, 0.001)

# 获取标定板角点的位置
objp = np.zeros((8 * 8, 3), np.float32)
objp[:, :2] = np.mgrid[0:8, 0:8].T.reshape(-1, 2)  # 将世界坐标系建在标定板上，所有点的Z坐标全部为0，所以只需要赋值x和y

# 连接到第一个可用的相机
camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())

# 获取图像并定义了转换器
camera.StartGrabbing(pylon.GrabStrategy_LatestImageOnly) 
converter = pylon.ImageFormatConverter()

# 将抓取到的图像转换成为opencv支持的图像格式
converter.OutputPixelFormat = pylon.PixelType_BGR8packed
converter.OutputBitAlignment = pylon.OutputBitAlignment_MsbAligned

num = 1

while True:
    # 拍摄数量达到上限之后自动退出
    num += 1
    if num == 100:
        break

    cnt = 0
    obj_points = []  # 存储3D点
    img_points = []  # 存储2D点
    if num % 10 == 0:
        break
    while camera.IsGrabbing():
        grabResult = camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)

        i = 0

        if grabResult.GrabSucceeded():
            # Access the image data
            image = converter.Convert(grabResult)
            img = image.GetArray()
            img = cv2.resize(img, None, fx=0.4, fy=0.4)

            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            size = gray.shape[::-1]
            ret, corners = cv2.findChessboardCorners(gray, (8, 8), None)

            if ret:

                obj_points.append(objp)

                corners2 = cv2.cornerSubPix(gray, corners, (5, 5), (-1, -1), criteria)  # 在原角点的基础上寻找亚像素角点
                if [corners2]:
                    img_points.append(corners2)
                else:
                    img_points.append(corners)

                cv2.drawChessboardCorners(img, (8, 8), corners, ret)  # 记住，OpenCV的绘制函数一般无返回值
                i+=1;
                cv2.imwrite('conimg'+str(i)+'.jpg', img)
                cv2.waitKey(1)

            cnt += 1

            if cnt % 2 == 0:
                break

        grabResult.Release()

    # 如果抓取到的图像中没有可以进行标定的棋盘格，则也退出
    if len(img_points) == 0:
        break

    # 标定
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points, size, None, None)

    # 重建立方体顶点的坐标
    origin = [[2, 2,  0], [2, 3,  0], [3, 3,  0], [3, 2,  0], [2, 2, -1], [2, 3, -1], [3, 3, -1], [3, 2, -1]]
    origin = np.array(origin)
    origin = np.hstack((origin, np.ones(8).reshape(8, 1)))

    # 罗德里格斯变换得到旋转矩阵
    rot = cv2.Rodrigues(rvecs[0])
    R = rot[0]
    # [R T]，3 x 4
    mat = np.hstack((R, tvecs[0].reshape((3, 1))))
    
    tmp1 = np.dot(mat, origin.T)
    out = np.dot(mtx, tmp1)
    out = out / out[2, :]

    draw_lines(img, out.T)

    cv2.imshow(str(i+1)+'.png', img)
    k = cv2.waitKey(1)
    if k == 27:
        break
            
# 重新释放资源 
camera.StopGrabbing()
camera.Close()
cv2.destroyAllWindows()